In [5]:
#hierarchical Clustering with MIN distance matrix updation method.
import numpy as np
import sys
from scipy.spatial import distance
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform


np.set_printoptions(suppress=True) #prevent numpy exponential 

def read_data(file):
    gene_data = open(file)
    gene_seq = gene_data.readlines()
    all_genes_list = []
    for line in gene_seq:
        gene = line.strip().split("\t")
        all_genes_list.append(gene)
    return np.asarray(all_genes_list, dtype = float)

def dist_matrix(data):
    distance_matrix = np.zeros([data.shape[0],data.shape[0]])
    distance_matrix = squareform(pdist(data, 'euclidean'))
    return distance_matrix

def perform_HAC(data, distance_matrix, cluster_count,clusters,iteration_count):
    print("it ",iteration_count , " clus cnt ",cluster_count)
#     if((iteration_count + cluster_count) == len(data)):
#         return clusters
    if(len(clusters) == cluster_count):
        return clusters
    
    minimum_dist = (distance_matrix[distance_matrix > 0]).min()
    closest_points = np.where(distance_matrix == minimum_dist)
    p1 = closest_points[1][0]
    p2 = closest_points[1][1]
    for g in range(0,len(data)):
        min_dis = min(distance_matrix[p1][g], distance_matrix[p2][g])
        distance_matrix[p1][g] = min_dis
        distance_matrix[g][p1] = min_dis
        distance_matrix[p2][g] = -1
        distance_matrix[g][p2] = -1
#     distance_matrix[p1][p2] = -1
#     distance_matrix[p2][p1] = -1
    for p2g in range(0,len(clusters[p2+1])):
        clusters[p1+1].append(clusters[p2+1][p2g])
    clusters.pop(p2+1)
    #clusters[p2+1] = clusters[p1+1]
    return perform_HAC(data, distance_matrix, cluster_count, clusters, iteration_count+1)

gene_data = read_data("cho.txt")
gene_features = gene_data[:,2:]
#print(gene_features)
distance_matrix = dist_matrix(gene_features)
cluster_count = int(input("Enter number of clusters: "))
clusters = dict()
for i in range(1,len(gene_data)+1):
    clusters[i] = [i-1]
#print(clusters)
clusters = perform_HAC(gene_features, distance_matrix, cluster_count,clusters,0)
print(len(clusters))
print(clusters)


Enter number of clusters: 6
it  0  clus cnt  6
it  1  clus cnt  6
it  2  clus cnt  6
it  3  clus cnt  6
it  4  clus cnt  6
it  5  clus cnt  6
it  6  clus cnt  6
it  7  clus cnt  6
it  8  clus cnt  6
it  9  clus cnt  6
it  10  clus cnt  6
it  11  clus cnt  6
it  12  clus cnt  6
it  13  clus cnt  6
it  14  clus cnt  6
it  15  clus cnt  6
it  16  clus cnt  6
it  17  clus cnt  6
it  18  clus cnt  6
it  19  clus cnt  6
it  20  clus cnt  6
it  21  clus cnt  6
it  22  clus cnt  6
it  23  clus cnt  6
it  24  clus cnt  6
it  25  clus cnt  6
it  26  clus cnt  6
it  27  clus cnt  6
it  28  clus cnt  6
it  29  clus cnt  6
it  30  clus cnt  6
it  31  clus cnt  6
it  32  clus cnt  6
it  33  clus cnt  6
it  34  clus cnt  6
it  35  clus cnt  6
it  36  clus cnt  6
it  37  clus cnt  6
it  38  clus cnt  6
it  39  clus cnt  6
it  40  clus cnt  6
it  41  clus cnt  6
it  42  clus cnt  6
it  43  clus cnt  6
it  44  clus cnt  6
it  45  clus cnt  6
it  46  clus cnt  6
it  47  clus cnt  6
it  48  clus cnt  6
it